<a href="https://colab.research.google.com/github/felipednegredo/tcc-emocoes-musicais-codigo/blob/main/TCC_Fingerprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q -U kaleido pyarrow fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00


## Teste 1

In [ ]:
# ===============================================================
#  FINGERPRINT em LOTE (blocks_by_song/*.parquet) — Shazam-like
#  - STFT (detecção de dB automático) + contraste estável
#  - Picos: densidade alvo + TOP-K por banda + equalização opcional
#  - Fan-out: âncora + linhas (sem Target Zone)  [sem círculo]
#  - Exporta: HTML por bloco e PARQUET com picos por música
#  - Fundo branco, ticks de X controlados, barras de progresso
#  - Ignora blocos com valence/arousal vazios
# ===============================================================

import os, re, io, zlib, math, time, warnings, pathlib
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import librosa
warnings.filterwarnings("ignore")

# ------------- tqdm (progress bar) -------------
try:
    from tqdm.auto import tqdm
except Exception:
    tqdm = None

try:
    from scipy.ndimage import median_filter
    SCIPY_OK = True
except Exception:
    SCIPY_OK = False

# ==================== CONFIG ====================
# Aceita diretório (varre *.parquet) ou um único arquivo
IN_PATH       = "/content/drive/MyDrive/DataSet TCC/DEAM/parquet/blocks_by_song"
OUT_ROOT      = "/content/drive/MyDrive/DataSet TCC/DEAM/fingerprint"

# Renderização (ambas independentes)
DO_DUAL       = True     # STFT | Constellation
DO_OVERLAY    = True     # STFT + Constellation

EXPORT_HTML   = True     # exporta HTML (sempre disponível)
IMG_W, IMG_H  = 1280, 640

# Visual
YLIM_HZ       = (50, 8000)
USE_LOG_Y     = False
SPEC_CMAP     = "Inferno"
ZLIM_FIXED    = (-75, 0)
TITLE_PREFIX  = "Fingerprint espectral — "

# Picos (bandas + TOP-K)
BANDS              = [(80,300), (300,1200), (1200,4000), (4000,8000)]
TOPK_PER_BAND      = 1

# Equalização para detecção (opcional)
EQUALIZE_FOR_PEAKS = True
EQ_SIZE_FREQ       = 15

# Controle de densidade (busca binária no percentil global)
AUTO_PEAK_DENSITY     = True
TARGET_PEAKS_PER_SEC  = 20.0
PCTL_SEARCH_RANGE     = (80.0, 99.9)
MAX_BINSEARCH_ITERS   = 12
TOLERANCE_RELATIVE    = 0.20

# Fan-out (âncora central + linhas) — sem círculo
SHOW_FAN_LINES  = True
FAN_T_MIN       = 0.02
FAN_T_MAX       = 0.50
FAN_FANOUT      = 6

# Eixo X (ticks em passo fixo)
X_TICK_STEP_BASE = 2.0  # segundos

# Símbolo dos picos: **bolinhas** (sem “X”)
PK_MARKER = dict(
    symbol="circle", size=6, opacity=0.95,
    color="#000000", line=dict(width=0.8, color="#111111")
)

# =================== HELPERS ====================
def ensure_dir(p): pathlib.Path(p).mkdir(parents=True, exist_ok=True)

def slugify(s):
    s = (s or "desconhecida").strip().lower()
    s = re.sub(r"[^\w\s-]+", "", s); s = re.sub(r"\s+", "_", s)
    return s[:60] or "sem_titulo"

def bytes_to_ndarray(b):
    if b is None: return np.array([])
    if isinstance(b, (bytes, bytearray)):
        try:
            raw = zlib.decompress(b)
            return np.load(io.BytesIO(raw), allow_pickle=False)
        except Exception:
            return np.load(io.BytesIO(b), allow_pickle=False)
    if isinstance(b, np.ndarray): return b
    return np.array([])

def _nice_tick_params(times: np.ndarray):
    if times is None or times.size == 0:
        return dict(dtick=X_TICK_STEP_BASE, tick0=0.0)
    t0 = float(np.nanmin(times)); t1 = float(np.nanmax(times))
    dur = max(t1 - t0, 0.0)
    step = X_TICK_STEP_BASE
    if dur > 60:   step = 5 * X_TICK_STEP_BASE
    if dur > 300:  step = 10 * X_TICK_STEP_BASE
    tick0 = math.floor(t0 / step) * step
    return dict(dtick=step, tick0=tick0)

def detect_scale_and_fix(S, freqs, times, zlim_fixed=ZLIM_FIXED):
    if S is None or S.size == 0:
        raise ValueError("detect_scale_and_fix: matriz S vazia.")
    S = np.array(S, copy=False)

    # Orientação
    try:
        if freqs.size and times.size and S.shape[0] == times.size and S.shape[1] == freqs.size:
            S = S.T
    except Exception:
        pass

    # Sanitização
    S = np.where(np.isfinite(S), S, np.nan)
    if np.isnan(S).all():
        raise ValueError("detect_scale_and_fix: S só contém NaN/Inf.")
    finite_min = float(np.nanmin(S))
    S = np.nan_to_num(S, nan=finite_min)

    # Escala
    S_min, S_max = float(np.min(S)), float(np.max(S))
    frac_neg = float((S < 0).mean()) if S.size else 0.0
    is_db_like = ((S_max <= 5.0 and S_min < -10.0) or (frac_neg > 0.6))

    if np.iscomplexobj(S):
        S_abs = np.abs(S); is_db_like = False
    else:
        S_abs = np.abs(S)

    if is_db_like:
        S_db = S.astype(float, copy=False)
    else:
        ref = np.max(S_abs) if np.max(S_abs) > 0 else 1.0
        S_db = librosa.amplitude_to_db(S_abs, ref=ref)

    # freqs/times default
    if freqs is None or (isinstance(freqs, np.ndarray) and freqs.size == 0):
        freqs = np.linspace(0.0, 22050.0, S_db.shape[0], dtype=float)
    if times is None or (isinstance(times, np.ndarray) and times.size == 0):
        times = np.arange(S_db.shape[1], dtype=float)

    # z-limits
    if zlim_fixed is not None and len(zlim_fixed) == 2:
        zmin, zmax = zlim_fixed
    else:
        zmin, zmax = np.nanpercentile(S_db, [1, 99])
    return S_db, np.asarray(freqs, float), np.asarray(times, float), (float(zmin), float(zmax))

def equalize_for_peaks(S_db):
    if not SCIPY_OK or EQ_SIZE_FREQ <= 1: return S_db
    return S_db - median_filter(S_db, size=(EQ_SIZE_FREQ, 1))

# -------- picos por percentil FIXO (base p/ busca binária) --------
def _peaks_with_percentile(S_db, freqs, times, pctl):
    if S_db is None or S_db.size == 0:
        return np.array([]), np.array([])
    freqs = np.asarray(freqs) if isinstance(freqs, np.ndarray) else np.array(freqs)
    times = np.asarray(times) if isinstance(times, np.ndarray) else np.array(times)
    if freqs.size == 0:
        freqs = np.linspace(0.0, 22050.0, S_db.shape[0], dtype=float)
    if times.size == 0:
        times = np.arange(S_db.shape[1], dtype=float)

    n_freq, n_frames = S_db.shape
    pk_t, pk_f = [], []

    S_for = equalize_for_peaks(S_db) if EQUALIZE_FOR_PEAKS else S_db
    thr_cols = np.percentile(S_for, pctl, axis=0)  # shape (n_frames,)

    for t in range(n_frames):
        col = S_for[:, t]
        thr = float(thr_cols[t])
        for (lo, hi) in BANDS:
            band = (freqs >= lo) & (freqs < hi)
            idxs = np.where((col >= thr) & band)[0]
            if idxs.size == 0:
                continue
            sel = idxs[np.argsort(col[idxs])[::-1][:TOPK_PER_BAND]]
            pk_t.extend([times[t]] * len(sel))
            pk_f.extend(freqs[sel].tolist())

    return np.asarray(pk_t), np.asarray(pk_f)

# -------- picos com densidade-alvo --------
def estimate_peaks_auto_density(S_db, freqs, times,
                                target_pps=TARGET_PEAKS_PER_SEC):
    if S_db is None or S_db.size == 0:
        return np.array([]), np.array([])
    freqs = np.asarray(freqs) if isinstance(freqs, np.ndarray) else np.array(freqs)
    times = np.asarray(times) if isinstance(times, np.ndarray) else np.array(times)
    if times.size == 0:
        times = np.arange(S_db.shape[1], dtype=float)

    dur = max(float(np.nanmax(times) - np.nanmin(times)), 1e-6)
    target_total = int(target_pps * dur)

    lo, hi = PCTL_SEARCH_RANGE
    best_pt, best_pf = np.array([]), np.array([]); best_diff = float("inf")

    for _ in range(MAX_BINSEARCH_ITERS):
        mid = (lo + hi) / 2.0
        pt, pf = _peaks_with_percentile(S_db, freqs, times, pctl=mid)
        n = int(pt.size)
        diff = abs(n - target_total)
        if diff < best_diff:
            best_diff, best_pt, best_pf = diff, pt, pf
        if target_total == 0 or abs(n - target_total) / max(target_total, 1) <= TOLERANCE_RELATIVE:
            break
        if n > target_total:
            lo = mid   # sobe percentil -> menos picos
        else:
            hi = mid   # desce percentil -> mais picos

    return best_pt, best_pf

def select_anchor_and_targets(pt, pf):
    if pt.size == 0 or pf.size == 0:
        return None, []
    idx_sorted = np.argsort(pt)
    i = idx_sorted[len(idx_sorted)//2]  # mediana temporal
    t_a, f_a = float(pt[i]), float(pf[i])
    mask = (pt > t_a + FAN_T_MIN) & (pt < t_a + FAN_T_MAX)
    cand = np.where(mask)[0]
    cand = cand[np.argsort(pt[cand])]
    lines = [(t_a, f_a, float(pt[j]), float(pf[j])) for j in cand[:FAN_FANOUT]]
    return (t_a, f_a), lines

def style_axes(fig, fmin, fmax, row=None, col=None, times=None):
    ykw = (dict(type="log", range=[math.log10(max(1, fmin)), math.log10(fmax)])
           if USE_LOG_Y else dict(type="linear", range=[fmin, fmax]))
    tk = _nice_tick_params(times)
    xkw = dict(showline=True, mirror=True, ticks="outside", showgrid=True,
               tickmode="linear", dtick=tk["dtick"], tick0=tk["tick0"], tickformat=".0f")
    fig.update_yaxes(**ykw, showgrid=True, zeroline=False, row=row, col=col)
    fig.update_xaxes(**xkw, row=row, col=col)

def add_titles(fig, main, sub):
    fig.update_layout(template="plotly_white", width=IMG_W, height=IMG_H,
                      paper_bgcolor="white", plot_bgcolor="white",
                      margin=dict(l=80, r=30, t=95, b=55))
    fig.add_annotation(text=main, xref="paper", yref="paper", x=0, y=1.15,
                       showarrow=False, font=dict(size=22, color="#1f2a44"))
    fig.add_annotation(text=sub, xref="paper", yref="paper", x=0, y=1.08,
                       showarrow=False, font=dict(size=14, color="#324a66"))

# ================== PLOT DE 1 BLOCO ==================
def plot_block(row, out_dir, dual=True, overlay=True):
    # blobs
    S_blk  = bytes_to_ndarray(row.get("stft_bytes"))
    freqs  = bytes_to_ndarray(row.get("freqs_bytes"))
    times  = bytes_to_ndarray(row.get("times_bytes"))
    if S_blk.size == 0: raise ValueError("stft_bytes vazio")

    # correções e recorte Y
    S_db, freqs, times, zlim = detect_scale_and_fix(S_blk, freqs, times)
    fmin, fmax = YLIM_HZ
    freqs = np.maximum(freqs, 0.0)
    band = (freqs >= fmin) & (freqs <= fmax)
    S_show, F_show = S_db[band, :], freqs[band]

    # picos
    if AUTO_PEAK_DENSITY:
        pt, pf = estimate_peaks_auto_density(S_db, freqs, times)
    else:
        pt, pf = _peaks_with_percentile(S_db, freqs, times, pctl=95.0)
    if pf.size:
        sel = (pf >= fmin) & (pf <= fmax)
        pf, pt = pf[sel], pt[sel]

    # metadados (copiados do parquet)
    title = row.get("title", "Desconhecida")
    val, aro = row.get("valence", None), row.get("arousal", None)
    emo = row.get("emotion_label", None)
    nfft = int(row.get("stft_n_fft", 2048) or 2048)
    hop  = int(row.get("stft_hop",   512)  or 512)
    win  = str(row.get("stft_window", "hann"))

    main = f"{TITLE_PREFIX}{title}"
    sub  = f"Bloco {int(row['block_idx'])} — STFT {win} {nfft}/{hop}"
    if pd.notna(val) and pd.notna(aro): sub += f" — Valência {val:.3f}, Arousal {aro:.3f}"
    if isinstance(emo, str) and emo:    sub += f" — Rótulo: {emo}"

    blk = int(row["block_idx"])
    base = os.path.join(out_dir, f"block_{blk:04d}")
    ensure_dir(out_dir)

    anchor_info, lines = select_anchor_and_targets(pt, pf)

    # Dual (sem círculo; mantém fan-lines se habilitadas)
    if dual:
        fig = make_subplots(rows=1, cols=2, column_widths=[0.6, 0.4], horizontal_spacing=0.08)
        fig.add_trace(go.Heatmap(z=S_show, x=times, y=F_show, colorscale=SPEC_CMAP,
                                 zmin=zlim[0], zmax=zlim[1],
                                 colorbar=dict(title="Magnitude (dB)", thickness=14, x=1.02)),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=pt, y=pf, mode="markers", marker=PK_MARKER, name="Picos"),
                      row=1, col=2)
        if anchor_info and SHOW_FAN_LINES:
            for (x0, y0, x1, y1) in lines:
                fig.add_shape(type="line", x0=x0, y0=y0, x1=x1, y1=y1,
                              line=dict(width=1, color="rgba(0,0,0,0.45)"), row=1, col=2)
        style_axes(fig, fmin, fmax, row=1, col=1, times=times)
        style_axes(fig, fmin, fmax, row=1, col=2, times=times)
        add_titles(fig, main, sub)
        if EXPORT_HTML:
            fig.write_html(base + "_dual.html", include_plotlyjs="cdn")

    # Overlay (sem círculo; mantém fan-lines se habilitadas)
    if overlay:
        fig2 = make_subplots(rows=1, cols=1)
        fig2.add_trace(go.Heatmap(z=S_show, x=times, y=F_show, colorscale=SPEC_CMAP,
                                  zmin=zlim[0], zmax=zlim[1],
                                  colorbar=dict(title="Magnitude (dB)", thickness=14, x=1.02)))
        fig2.add_trace(go.Scatter(x=pt, y=pf, mode="markers", marker=PK_MARKER, name="Picos"))
        if anchor_info and SHOW_FAN_LINES:
            for (x0, y0, x1, y1) in lines:
                fig2.add_shape(type="line", x0=x0, y0=y0, x1=x1, y1=y1,
                               line=dict(width=1, color="rgba(0,0,0,0.45)"))
        style_axes(fig2, fmin, fmax, row=1, col=1, times=times)
        add_titles(fig2, main, sub + " — Overlay")
        if EXPORT_HTML:
            fig2.write_html(base + "_overlay.html", include_plotlyjs="cdn")

    # DF de picos (inclui VA/label do bloco p/ validação)
    n = len(pt)
    return pd.DataFrame({
        "song_id":       [row.get("song_id")] * n,
        "block_idx":     [blk] * n,
        "time_s":        pt.astype(float) if n else np.array([], dtype=float),
        "freq_hz":       pf.astype(float) if n else np.array([], dtype=float),
        "title":         [title] * n,
        "valence":       [val] * n,
        "arousal":       [aro] * n,
        "emotion_label": [emo] * n
    })

# ================ UTIL DE I/O ================
def _iter_song_parquets(in_path):
    p = pathlib.Path(in_path)
    if p.is_dir():
        files = sorted(p.glob("song_*.parquet"))
    else:
        files = [p]
    out = []
    for fp in files:
        m = re.search(r"song_(\d+)\.parquet$", fp.name)
        sid = m.group(1) if m else fp.stem
        out.append((str(fp), sid))
    return out

# ================ LOOP PRINCIPAL ================
def process_all_songs():
    songs = _iter_song_parquets(IN_PATH)
    if not songs:
        print(f"[ERRO] Nenhum parquet encontrado em: {IN_PATH}")
        return

    song_iter = tqdm(songs, desc="Songs", unit="song") if tqdm else songs
    all_index_rows = []

    for song_path, song_id_str in song_iter:
        try:
            df = pd.read_parquet(song_path)
        except Exception as e:
            print(f"\n[ERRO] Song {song_id_str}: falha ao ler parquet: {e}")
            continue

        # normaliza/ordena
        if "block_idx" in df.columns:
            df["block_idx"] = pd.to_numeric(df["block_idx"], errors="coerce").astype("Int64")
            df = df.sort_values("block_idx").reset_index(drop=True)

        # ---- NOVO: ignorar blocos sem valence/arousal
        if {"valence","arousal"}.issubset(df.columns):
            df = df[df["valence"].notna() & df["arousal"].notna()].copy()
        else:
            print(f"[WARN] Song {song_id_str}: colunas valence/arousal ausentes — pulando música.")
            continue

        if df.empty:
            print(f"[WARN] Song {song_id_str}: todos os blocos sem VA — pulando música.")
            continue

        # pasta por música
        try:
            song_id  = int(df.loc[df["block_idx"].idxmin(), "song_id"]) if "song_id" in df else int(song_id_str)
        except Exception:
            song_id = int(song_id_str) if str(song_id_str).isdigit() else 0
        song_tit = str(df.loc[df["block_idx"].idxmin(), "title"]) if "title" in df else "Desconhecida"
        song_dir = os.path.join(OUT_ROOT, f"{song_id:04d}_{slugify(song_tit)}")
        ensure_dir(song_dir)

        # progresso por blocos
        blocks = list(df.itertuples(index=False))
        blk_iter = tqdm(blocks, desc=f"Song {song_id_str}", unit="blk", leave=False) if tqdm else blocks

        peaks_parts = []
        for row in blk_iter:
            row = row._asdict() if hasattr(row, "_asdict") else row
            try:
                dfp = plot_block(row, song_dir, dual=DO_DUAL, overlay=DO_OVERLAY)
                peaks_parts.append(dfp)
            except Exception as e:
                print(f"\n[ERRO] Song {song_id_str} bloco {row.get('block_idx')}: {e}")

        # salvar parquet de picos da música
        if peaks_parts:
            peaks_df = pd.concat(peaks_parts, ignore_index=True)
            out_pq   = os.path.join(song_dir, "peaks_all_blocks.parquet")
            peaks_df.to_parquet(out_pq, index=False, compression="snappy")
            all_index_rows.append(dict(song_id=song_id, title=song_tit, peaks_path=out_pq,
                                       n_points=len(peaks_df)))
        else:
            print(f"[WARN] Song {song_id_str}: nenhum pico coletado.")

    # índice geral (opcional)
    if all_index_rows:
        idx_df = pd.DataFrame(all_index_rows).sort_values("song_id")
        ensure_dir(OUT_ROOT)
        idx_path = os.path.join(OUT_ROOT, "peaks_index.parquet")
        idx_df.to_parquet(idx_path, index=False, compression="snappy")
        print(f"\n[OK] Índice de picos salvo em: {idx_path}")
    print("[DONE] Lote concluído.")

# ---- run ----
process_all_songs()


In [ ]:
# -*- coding: utf-8 -*-
# ===============================================================
#  EMOTION FINGERPRINT INDEXED (TCC — busca por emoções)
#  + LOGS DETALHADOS DE EXECUÇÃO
# ===============================================================

from __future__ import annotations
import os, math, warnings, time, re, unicodedata   # <-- add re, unicodedata
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any, Optional

import numpy as np
import pandas as pd
import librosa as lb

warnings.filterwarnings("ignore")

try:
    from tqdm.auto import tqdm
except Exception:
    tqdm = None

# ===============================================================
# CONFIGURAÇÕES
# ===============================================================

PEAKS_INDEX_PARQUET = "/content/drive/MyDrive/DataSet TCC/DEAM/fingerprint/peaks_index.parquet"

@dataclass
class FPConfig:
    n_fft: int = 2048
    hop_length: int = 512
    window: str = "hann"
    center: bool = True
    to_db: bool = True
    topk_per_col: int = 5
    fan_out: int = 5
    min_dt: float = 0.02
    max_dt: float = 3.0
    min_df: float = 50.0
    sr: int = 22050


# ===============================================================
# STFT PADRONIZADA (para fingerprint)
# ===============================================================
def compute_stft(y: np.ndarray, sr: int, n_fft: int = 2048, hop_length: int = 512,
                 window: str = "hann", center: bool = True, to_db: bool = True) -> Dict[str, Any]:
    S = lb.stft(y, n_fft=n_fft, hop_length=hop_length, window=window, center=center)
    mag = np.abs(S)
    if to_db:
        mag = lb.amplitude_to_db(mag, ref=np.max)
    times = lb.frames_to_time(np.arange(mag.shape[1]), sr=sr, hop_length=hop_length, n_fft=n_fft)
    freqs = lb.fft_frequencies(sr=sr, n_fft=n_fft)
    meta = dict(sr=sr, n_fft=n_fft, hop_length=hop_length, window=window, center=center, scale_db=to_db)
    return {"spec": mag, "times": times, "freqs": freqs, "meta": meta}


# ===============================================================
# CARREGAMENTO E FILTRAGEM
# ===============================================================
def load_index(path=PEAKS_INDEX_PARQUET) -> pd.DataFrame:
    print(f"\n📂 Carregando índice global de picos: {path}")
    df = pd.read_parquet(path)
    expected = ["song_id", "title", "peaks_path", "n_points"]
    missing = [c for c in expected if c not in df.columns]
    if missing:
        raise ValueError(f"Colunas ausentes no índice: {missing}")
    df["song_id"] = df["song_id"].astype("Int32")
    df["title"] = df["title"].astype(str)
    df["peaks_path"] = df["peaks_path"].astype(str)
    df["n_points"] = pd.to_numeric(df["n_points"], errors="coerce").fillna(0).astype(int)
    print(f"✅ Índice carregado com {len(df)} músicas.")
    return df


def load_peaks_from_path(peaks_path: str, columns: Optional[List[str]] = None) -> pd.DataFrame:
    if not os.path.exists(peaks_path):
        raise FileNotFoundError(f"Arquivo de picos não encontrado: {peaks_path}")
    return pd.read_parquet(peaks_path, columns=columns)


# ===============================================================
# FINGERPRINT (Shazam-like)
# ===============================================================
def detect_peaks_constellation(S_db: np.ndarray, cfg: FPConfig) -> List[Tuple[int, int]]:
    H, W = S_db.shape
    peaks = []
    for x in range(W):
        col = S_db[:, x]
        idx = np.argpartition(col, -cfg.topk_per_col)[-cfg.topk_per_col:]
        idx = idx[np.argsort(col[idx])][::-1]
        for fbin in idx:
            peaks.append((x, fbin))
    return peaks


def pairs_to_hashes(peaks: List[Tuple[int, int]], cfg: FPConfig, sr: int) -> List[Tuple[int, float, int, float, int]]:
    hashes = []
    hop_t = cfg.hop_length / sr
    hz_per_bin = (sr / cfg.n_fft) / 2.0
    for i, (xa, fa) in enumerate(peaks):
        for j in range(1, cfg.fan_out + 1):
            if i + j >= len(peaks): break
            xt, ft = peaks[i + j]
            dt = (xt - xa) * hop_t
            df = (ft - fa) * hz_per_bin
            if dt < cfg.min_dt or dt > cfg.max_dt: continue
            if abs(df) < cfg.min_df: continue
            q_dt = int(round(dt * 10))
            q_df = int(round(df / 20.0))
            h = (q_df & 0xFFFF) << 16 | (q_dt & 0xFFFF)
            hashes.append((h, xa * hop_t, fa, xt * hop_t, ft))
    return hashes


def fingerprint_wav(path: str, cfg=FPConfig()) -> pd.DataFrame:
    print(f"\n🎧 Gerando fingerprint do clipe: {os.path.basename(path)}")
    y, sr = lb.load(path, sr=cfg.sr, mono=True)
    print(f"   - Duração: {len(y)/sr:.2f}s  SR={sr}")
    stft_out = compute_stft(y, sr, n_fft=cfg.n_fft, hop_length=cfg.hop_length,
                            window=cfg.window, center=cfg.center, to_db=cfg.to_db)
    S_db = stft_out["spec"]
    peaks = detect_peaks_constellation(S_db, cfg)
    hashes = pairs_to_hashes(peaks, cfg, sr)
    print(f"   - {len(peaks)} picos detectados → {len(hashes)} hashes gerados")
    return pd.DataFrame(hashes, columns=["hash", "tq", "fq", "tq2", "fq2"])


# ===============================================================
# BUSCA (com LOGS) — AGORA COM EMOÇÕES COMPOSTAS/SINÔNIMOS
# ===============================================================
def _normalize_text(text: str) -> str:
    """remove acentos e coloca em minúsculas."""
    text = unicodedata.normalize("NFKD", str(text))
    text = "".join(c for c in text if not unicodedata.combining(c))
    return text.lower().strip()

# mapeamento opcional de sinônimos → rótulo canônico (ajuste ao seu índice)
EMOTION_SYNONYMS = {
    # quadrante excitado/positivo
    "excitante": "excitado_positivo",
    "positivo":  "excitado_positivo",
    "feliz":     "excitado_positivo",
    "euforico":  "excitado_positivo",
    "energetico":"excitado_positivo",
    "animado":   "excitado_positivo",
    "alegre":    "excitado_positivo",
    # você pode expandir com outros quadrantes aqui...
}

def _expand_emotion_keywords(emotion: str) -> list[str]:
    """
    Divide a emoção composta e normaliza:
    'Excitante/positivo (feliz, eufórico, enérgico)'
      → ['excitante','positivo','feliz','euforico','energetico']
    """
    base = _normalize_text(emotion)
    parts = re.split(r"[/,;()]+", base)
    parts = [p.strip() for p in parts if p.strip()]
    # substitui por canônicos quando houver
    out = []
    for p in parts:
        out.append(EMOTION_SYNONYMS.get(p, p))
    # remove duplicatas preservando ordem
    seen, uniq = set(), []
    for p in out:
        if p not in seen:
            seen.add(p)
            uniq.append(p)
    return uniq

def search_clip(peaks_index: pd.DataFrame,
                emotion: Optional[str] = None,
                prefilter_top: int = 50,
                results_topn: int = 10) -> pd.DataFrame:
    """
    Busca emocional no índice de fingerprints (sem clipe externo).
    Agora aceita emoções compostas/sinônimos.
    """
    print("\n🔍 Iniciando busca no índice de fingerprints...")
    start_time = time.time()

    # Fase 0 — Filtragem emocional (composta/sinônimos)
    if emotion and "emotion_zone" in peaks_index.columns:
        keys = _expand_emotion_keywords(emotion)
        print(f"🎭 Emoção solicitada : {emotion}")
        print(f"🔹 Palavras/labels  : {keys}")
        col_norm = peaks_index["emotion_zone"].astype(str).apply(_normalize_text)

        # monta máscara: casa se QUALQUER termo aparecer
        mask = pd.Series(False, index=peaks_index.index)
        for kw in keys:
            # procura por palavra inteira OU rótulo canônico
            mask |= col_norm.str.contains(rf"\b{re.escape(kw)}\b", regex=True)
        df = peaks_index.loc[mask].copy()
        print(f"🎯 {len(df)} músicas encontradas com pelo menos um termo.")
    else:
        df = peaks_index.copy()
        print(f"🎭 Nenhum filtro emocional aplicado ({len(df)} músicas no total).")

    if df.empty:
        print("❌ Nenhum resultado encontrado no índice.")
        return pd.DataFrame()

    # Fase 1 — Ordenação preliminar (densidade de picos)
    df = df.sort_values("n_points", ascending=False).head(prefilter_top)
    print(f"📊 Selecionando Top-{prefilter_top} por número de picos (densidade espectral).")

    # Fase 2 — Estatística simples
    avg_points = df["n_points"].mean()
    print(f"📈 Média de picos nas músicas selecionadas: {avg_points:.1f}")

    # Fase 3 — Ranking final
    df_final = df.head(results_topn).reset_index(drop=True)
    dur = time.time() - start_time
    print(f"\n🏁 Busca concluída em {dur:.2f}s — {len(df_final)} melhores resultados:\n")
    cols_show = ["song_id", "title", "n_points", "emotion_zone"] if "emotion_zone" in df_final.columns else ["song_id","title","n_points"]
    print(df_final[cols_show])

    return df_final


# ===============================================================
# PLOT PARA INSERIR NO TCC
# ===============================================================
def plot_constellation(peaks_path: str, title: str = "", max_points=2000):
    import plotly.graph_objects as go
    df = load_peaks_from_path(peaks_path)
    if "t_anchor" not in df.columns or "f_anchor" not in df.columns:
        raise ValueError("Arquivo sem colunas t_anchor/f_anchor")
    if len(df) > max_points:
        df = df.sample(max_points, random_state=0)
    fig = go.Figure(go.Scattergl(x=df["t_anchor"], y=df["f_anchor"],
                                 mode="markers", marker=dict(size=3)))
    fig.update_layout(template="plotly_white", width=900, height=420,
                      xaxis_title="Tempo (s)", yaxis_title="Frequência (Hz)",
                      title=f"Constelação — {title or os.path.basename(peaks_path)}")
    return fig


# ===============================================================
# EXEMPLO DE EXECUÇÃO
# ===============================================================
if __name__ == "__main__":
    index_df = load_index(PEAKS_INDEX_PARQUET)
    print(f"\n🔢 Total de músicas no índice: {len(index_df)}")

    # Exemplo 1 — emoção composta com sinônimos:
    results = search_clip(
        index_df,
        emotion="Excitante/positivo (feliz, eufórico, enérgico)",
        results_topn=10
    )

    # Exemplo 2 — sem filtro emocional:
    # results = search_clip(index_df)


## Teste 2

### Config

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
import numpy as np, pandas as pd, hashlib

IN_BLOCKS_DIR  = Path("/content/drive/MyDrive/DataSet TCC/DEAM/parquet/blocks_by_song")
OUT_FINGER_DIR = Path("/content/drive/MyDrive/DataSet TCC/DEAM/fingerprint")

OUT_FINGER_DIR.mkdir(parents=True, exist_ok=True)

PEAKS_INDEX_PATH = OUT_FINGER_DIR / "peaks_index.parquet"
REPORTS_DIR      = OUT_FINGER_DIR / "_reports"
REPORTS_DIR.mkdir(exist_ok=True, parents=True)

### VA

In [ ]:
# Quadrantes (Russell): Eixo X = Valence, Eixo Y = Arousal
QUADRANT_NAMES = {
    "Q1": "Feliz/Excitado (V+ A+)",   # high valence, high arousal
    "Q2": "Tenso/Ansioso (V- A+)",    # low valence, high arousal
    "Q3": "Triste/Letárgico (V- A-)", # low valence, low arousal
    "Q4": "Calmo/Relaxado (V+ A-)",   # high valence, low arousal
}

# Limiares de z-score ou escala normalizada (ajuste se usa VA z-score ou VA [0,1])
# Aqui assumo VA centrado em 0; mude para seus thresholds reais.
VALENCE_TH = 0.0
AROUSAL_TH = 0.0

def va_to_quadrant(valence: float, arousal: float) -> str:
    if pd.isna(valence) or pd.isna(arousal):
        return "NA"
    if valence >= VALENCE_TH and arousal >= AROUSAL_TH:
        return "Q1"
    if valence < VALENCE_TH and arousal >= AROUSAL_TH:
        return "Q2"
    if valence < VALENCE_TH and arousal < AROUSAL_TH:
        return "Q3"
    return "Q4"

# ----- DICIONÁRIO DE SINÔNIMOS (PT/EN) POR QUADRANTE -----
EMO_SYNONYMS = {
    "Q1": [
        "feliz","eufórico","animado","energético","empolgado","alegre",
        "happy","excited","joyful","energetic","upbeat","elated"
    ],
    "Q2": [
        "tenso","ansioso","nervoso","raiva","irritado","intenso",
        "tense","anxious","angry","irritated","agitated","stressed"
    ],
    "Q3": [
        "triste","depressivo","melancólico","abatido","letárgico",
        "sad","depressed","melancholic","blue","down","lethargic"
    ],
    "Q4": [
        "calmo","relaxado","tranquilo","sereno","confortável","suave",
        "calm","relaxed","peaceful","soothing","chill","serene"
    ]
}

def expand_emotion_query(q: str) -> set:
    """Recebe um termo (ex.: 'feliz') e devolve o conjunto de termos equivalentes nos 4 quadrantes."""
    qnorm = (q or "").strip().lower()
    hits = set()
    for quad, words in EMO_SYNONYMS.items():
        for w in words:
            if qnorm in w or w in qnorm:
                hits.update(words)
    # Se não bateu sinônimo algum, considere que o usuário quer string literal:
    if not hits and qnorm:
        hits.add(qnorm)
    return hits

def mask_by_emotion_zone(df: pd.DataFrame, query: str) -> pd.Series:
    """Filtra por emotion_zone (texto) usando sinônimos expansíveis."""
    if "emotion_zone" not in df.columns:
        return pd.Series([True] * len(df), index=df.index)  # sem coluna, não filtra
    synonyms = expand_emotion_query(query)
    if not synonyms:
        return pd.Series([True] * len(df), index=df.index)
    s = df["emotion_zone"].fillna("").str.lower()
    mask = s.apply(lambda x: any(term in x for term in synonyms))
    return mask

def plot_va_quadrants(index_df: pd.DataFrame, title="Mapa Valence–Arousal (Exemplos)"):
    """
    Plota VA do índice global, colorindo por quadrante.
    Espera colunas: 'valence', 'arousal' no index_df (p.ex., médias por música).
    """
    df = index_df.copy()
    if "valence" not in df.columns or "arousal" not in df.columns:
        raise ValueError("index_df precisa ter colunas 'valence' e 'arousal'.")

    df["quadrant"] = [va_to_quadrant(v, a) for v, a in zip(df["valence"], df["arousal"])]

    color_map = {"Q1":"#2ca02c","Q2":"#d62728","Q3":"#1f77b4","Q4":"#ff7f0e","NA":"#7f7f7f"}
    fig = go.Figure()
    for qk, qname in QUADRANT_NAMES.items():
        sub = df[df["quadrant"] == qk]
        if len(sub)==0:
            continue
        fig.add_scatter(
            x=sub["valence"], y=sub["arousal"], mode="markers",
            name=qname, marker=dict(size=8, opacity=0.75),
            marker_color=color_map.get(qk, "#7f7f7f"),
            text=sub.get("title", "")
        )

    # Eixos e cruz central (0,0)
    fig.update_layout(
        title=title, xaxis_title="Valence", yaxis_title="Arousal",
        width=800, height=600, template="plotly_white"
    )
    fig.add_hline(y=AROUSAL_TH, line_width=1, line_dash="dot", line_color="#444")
    fig.add_vline(x=VALENCE_TH, line_width=1, line_dash="dot", line_color="#444")
    return fig

### Funções de hashing e salvamento

In [ ]:
# --- EXEMPLO de quantização usada no seu pipeline (ajuste se precisar) ---
def _q(v: float, step: float) -> int:
    return int(np.round(v / step))

def make_pair_hash(f_anchor_hz: float, f_target_hz: float,
                   dt_sec: float, df_hz: float,
                   q_dt: float, q_df: float, q_f: float) -> str:
    """
    Gera um hash curto e estável a partir dos valores quantizados.
    """
    fa_q  = _q(f_anchor_hz, q_f)
    ft_q  = _q(f_target_hz, q_f)
    dt_q  = _q(dt_sec,      q_dt)
    df_q  = _q(df_hz,       q_df)

    # Canonical tuple → bytes → blake2 (rápido, estável)
    payload = f"{fa_q}|{ft_q}|{dt_q}|{df_q}".encode("utf-8")
    h = hashlib.blake2b(payload, digest_size=8).hexdigest()  # 16 hex chars
    return h

def attach_uniqueness_keys(peaks_pairs_df: pd.DataFrame,
                           song_id, block_idx) -> pd.DataFrame:
    """
    Garante chaves explícitas e verificáveis por linha de par âncora–alvo.
    Supõe colunas: t_anchor, f_anchor, t_target, f_target, hash.
    """
    df = peaks_pairs_df.copy()
    df["song_id"]   = song_id
    df["block_idx"] = block_idx

    # UID explícido (não substitui 'hash'); ajuda a depurar colisões entre blocos
    uid_str = (df["song_id"].astype(str) + "|" +
               df["block_idx"].astype(str) + "|" +
               df["t_anchor"].round(4).astype(str) + "|" +
               df["f_anchor"].round(1).astype(str) + "|" +
               df["t_target"].round(4).astype(str) + "|" +
               df["f_target"].round(1).astype(str) + "|" +
               df["hash"].astype(str))
    df["uid"] = uid_str.apply(lambda s: hashlib.blake2b(s.encode("utf-8"), digest_size=10).hexdigest())
    return df

def persist_peaks_with_stft_meta(peaks_df: pd.DataFrame,
                                 out_parquet_path,
                                 stft_meta: dict):
    """
    Persiste picos + METADADOS STFT como colunas (stft_nfft, stft_hop, stft_window, sr, etc.).
    """
    df = peaks_df.copy()
    # Metadados que queremos materializar como colunas
    for k, v in stft_meta.items():
        df[k] = v
    # Ex.: também persistir limites gráficos/peaks config:
    defaults = dict(
        spec_ylim_low=50, spec_ylim_high=8000,
        zlim_low=-75, zlim_high=0,
        peak_topk=stft_meta.get("peak_topk", None),
        peak_target_per_sec=stft_meta.get("target_peaks_per_sec", None),
        fanout=stft_meta.get("fan_out", None),
        dt_min=stft_meta.get("dt_min", None),
        dt_max=stft_meta.get("dt_max", None),
        df_min=stft_meta.get("df_min", None),
    )
    for k, v in defaults.items():
        if k not in df.columns:
            df[k] = v
    df.to_parquet(out_parquet_path, index=False)
    return out_parquet_path

### Colisões

In [ ]:
def collision_report(peaks_parquet_path: Path, out_csv: Path):
    """
    Reporta colisões de hash dentro de um arquivo de picos por música,
    considerando chaves (hash) que aparecem em múltiplos blocos.
    """
    df = pd.read_parquet(peaks_parquet_path)
    if not {"hash","block_idx"}.issubset(df.columns):
        raise ValueError("Parquet sem colunas necessárias: 'hash', 'block_idx'")

    # quantos blocos diferentes compartilham o mesmo hash?
    g = df.groupby("hash")["block_idx"].nunique().reset_index(name="n_blocks")
    collisions = g[g["n_blocks"] > 1].copy()

    # Exibir amostras detalhadas:
    details = None
    if len(collisions):
        details = (
            df.merge(collisions[["hash"]], on="hash", how="inner")
              .sort_values(["hash","block_idx","t_anchor"])
        )

    # Salva CSV com estatísticas
    stats = {
        "total_pairs": len(df),
        "unique_hashes": df["hash"].nunique(),
        "colliding_hashes": int((g["n_blocks"] > 1).sum()),
    }
    pd.DataFrame([stats]).to_csv(out_csv.with_name(out_csv.stem + "_stats.csv"), index=False)
    collisions.to_csv(out_csv, index=False)
    if details is not None:
        details.to_csv(out_csv.with_name(out_csv.stem + "_details.csv"), index=False)
    return stats

### Testes Hash

In [ ]:
def augment_audio(y, sr, time_stretch=1.0, pitch_shift=0.0):
    z = y
    if time_stretch and time_stretch != 1.0:
        z = librosa.effects.time_stretch(z, rate=time_stretch)
    if pitch_shift and pitch_shift != 0.0:
        z = librosa.effects.pitch_shift(z, sr=sr, n_steps=pitch_shift)
    return z

def fingerprint_match_rate(ref_pairs: pd.DataFrame, test_pairs: pd.DataFrame) -> float:
    """
    Mede interseção de 'hash' entre conjunto referência e teste.
    Simples, mas útil para ver sensibilidade a stretch/shift.
    """
    if "hash" not in ref_pairs.columns or "hash" not in test_pairs.columns:
        return 0.0
    a = set(ref_pairs["hash"].values.tolist())
    b = set(test_pairs["hash"].values.tolist())
    if not a:
        return 0.0
    return len(a & b) / len(a)

def run_robustness_grid(y, sr, fp_func, grid=None):
    """
    Executa uma grade de augmentations e mede match_rate.
    - fp_func(y, sr) deve retornar DataFrame com coluna 'hash'.
    - grid: lista de (time_stretch, pitch_shift)
    """
    if grid is None:
        grid = [(1.0,0.0), (0.9,0.0), (1.1,0.0), (1.0,1.0), (1.0,-1.0), (0.9,1.0), (1.1,-1.0)]
    ref_df = fp_func(y, sr)  # baseline
    results = []
    for ts, ps in grid:
        y_aug = augment_audio(y, sr, time_stretch=ts, pitch_shift=ps)
        test_df = fp_func(y_aug, sr)
        mr = fingerprint_match_rate(ref_df, test_df)
        results.append({"time_stretch": ts, "pitch_shift": ps, "match_rate": mr})
    return pd.DataFrame(results)

In [ ]:
def build_index():
    rows = []
    for p in OUT_FINGER_DIR.glob("*_*/peaks_all_blocks.parquet"):
        try:
            df = pd.read_parquet(p)
        except Exception as e:
            print(f"[WARN] Falha lendo {p}: {e}")
            continue
        song_id = df["song_id"].iloc[0] if "song_id" in df.columns else None
        title   = df["title"].iloc[0]    if "title"   in df.columns else None

        # Agregações úteis:
        n_pairs = len(df)
        n_hash  = df["hash"].nunique() if "hash" in df.columns else None

        # Média VA por música (se vier por bloco, na ausência de outra tabela canônica)
        valence = df["valence"].dropna().mean() if "valence" in df.columns else None
        arousal = df["arousal"].dropna().mean() if "arousal" in df.columns else None

        rows.append(dict(
            song_id=song_id, title=title, peaks_path=str(p),
            n_pairs=n_pairs, n_hash=n_hash,
            valence=valence, arousal=arousal
        ))
    idx = pd.DataFrame(rows)
    if len(idx)==0:
        print("[WARN] Nada encontrado em", OUT_FINGER_DIR)
        return
    idx.sort_values(["title","song_id"], inplace=True)
    idx.to_parquet(PEAKS_INDEX_PATH, index=False)
    print("✔ Index salvo em:", PEAKS_INDEX_PATH)

if __name__ == "__main__":
    build_index()

In [ ]:
def validate_blocks():
    issues = []
    for pq in IN_BLOCKS_DIR.glob("song_*.parquet"):
        try:
            df = pd.read_parquet(pq, columns=["song_id","block_idx","valence","arousal"])
        except Exception as e:
            issues.append({"path": str(pq), "error": str(e), "issue": "read_error"})
            continue

        if len(df)==0:
            issues.append({"path": str(pq), "error": "", "issue": "empty_file"})
            continue

        missing_v = df["valence"].isna().sum() if "valence" in df.columns else len(df)
        missing_a = df["arousal"].isna().sum() if "arousal" in df.columns else len(df)

        if missing_v>0 or missing_a>0 or "valence" not in df.columns or "arousal" not in df.columns:
            issues.append({
                "path": str(pq),
                "error": "",
                "issue": "missing_valence_arousal",
                "n_rows": len(df),
                "missing_valence": int(missing_v),
                "missing_arousal": int(missing_a),
            })

    rep = pd.DataFrame(issues)
    out = REPORTS_DIR / "validate_blocks_report.csv"
    rep.to_csv(out, index=False)
    print("✔ Relatório salvo em:", out)
    print(rep.head())

if __name__ == "__main__":
    validate_blocks()

In [ ]:
def _gallery_for_song(peaks_path: Path, n_blocks=4, max_points=2000):
    df = pd.read_parquet(peaks_path)
    if "block_idx" not in df.columns:
        return None

    blocks = sorted(df["block_idx"].dropna().unique().tolist())
    if not blocks:
        return None
    sample_blocks = random.sample(blocks, min(n_blocks, len(blocks)))

    fig = go.Figure()
    for b in sample_blocks:
        sub = df[df["block_idx"] == b]
        sub = sub.sample(n=min(max_points, len(sub)), random_state=42) if len(sub) > max_points else sub
        fig.add_scattergl(
            x=sub["t_anchor"], y=sub["f_anchor"], mode="markers",
            name=f"block {b}", marker=dict(size=3), opacity=0.7
        )
    fig.update_layout(template="plotly_white", height=500, width=800,
                      xaxis_title="Tempo (s)", yaxis_title="Frequência (Hz)",
                      title=f"Constellation — {peaks_path.parent.name}")
    return fig

def make_quadrant_galleries(index_path: Path, out_dir: Path, n_per_quad=4):
    idx = pd.read_parquet(index_path)
    if not {"valence","arousal","peaks_path"}.issubset(idx.columns):
        raise ValueError("index precisa ter colunas 'valence', 'arousal', 'peaks_path'.")

    idx["quadrant"] = [va_to_quadrant(v, a) for v, a in zip(idx["valence"], idx["arousal"])]

    for qk, qname in QUADRANT_NAMES.items():
        sub = idx[idx["quadrant"] == qk]
        if len(sub)==0:
            print(f"[INFO] Sem itens para {qname}")
            continue
        osel = sub.sample(n=min(n_per_quad, len(sub)), random_state=123)
        for _, row in osel.iterrows():
            fig = _gallery_for_song(Path(row["peaks_path"]))
            if fig is None:
                continue
            out_html = out_dir / f"gallery_{qk}_{Path(row['peaks_path']).parent.name}.html"
            fig.write_html(out_html, include_plotlyjs="cdn")
            print("✔", out_html)

if __name__ == "__main__":
    from common_paths import PEAKS_INDEX_PATH
    make_quadrant_galleries(PEAKS_INDEX_PATH, REPORTS_DIR, n_per_quad=3)

In [ ]:
# Exemplo de metadados STFT e config de peaks a persistir
stft_meta = dict(
    stft_nfft=2048,
    stft_hop=512,
    stft_window="hann",
    sr=22050,
    peak_topk=5,                    # ou seu valor real
    target_peaks_per_sec=300,       # se você usa densidade alvo
    fan_out=5,
    dt_min=0.02, dt_max=0.5,        # em segundos
    df_min=40.0,                    # em Hz
    q_dt=0.01, q_df=20.0, q_f=20.0  # passos de quantização usados no make_pair_hash
)

# Depois de gerar os pares (com colunas t_anchor, f_anchor, t_target, f_target):
pairs["hash"] = [
    make_pair_hash(fa, ft, (tt-ta), (ft-fa), stft_meta["q_dt"], stft_meta["q_df"], stft_meta["q_f"])
    for fa, ft, ta, tt in zip(pairs["f_anchor"], pairs["f_target"], pairs["t_anchor"], pairs["t_target"])
]

pairs = attach_uniqueness_keys(pairs, song_id=row["song_id"], block_idx=row["block_idx"])

# Adicione colunas de valence/arousal do bloco (se existirem no DF do bloco original)
for col in ("valence", "arousal", "emotion_zone", "title"):
    if col in row and col not in pairs.columns:
        pairs[col] = row[col]

persist_peaks_with_stft_meta(pairs, out_parquet_path, stft_meta)

# Opcional: após salvar cada música, emita relatório de colisões
stats = collision_report(out_parquet_path, out_parquet_path.with_name("hash_collisions.csv"))
print("Collision stats:", stats)


In [ ]:
import pandas as pd
import plotly.express as px
from pathlib import Path

# Caminho do seu parquet (ajuste para o do seu Drive)
PARQUET_PATH = "/content/drive/MyDrive/DataSet TCC/DEAM/fingerprint/0646_Backed_Vibes_Clean/peaks_all_blocks.parquet"

# Nomes de colunas "preferidos" (se existirem)
COL_SONG_CANDIDATES        = ["song_id", "track_id", "id", "deam_id"]
COL_TITLE_CANDIDATES       = ["title", "song_title", "track_title", "music_title"]
COL_BLOCK_CANDIDATES       = ["block_id", "segment_id", "block"]

COL_T_ANCHOR_CANDIDATES    = ["t_anchor_s", "t_anchor", "t_s", "time_s", "time", "t"]
COL_F_ANCHOR_CANDIDATES    = ["f_anchor_hz", "f_anchor", "f_hz", "freq_hz", "frequency", "f"]

COL_EMOTION_CANDIDATES     = ["emo_quadrant", "emotion_quadrant", "emo_label", "emotion_label"]

COL_MAG_CANDIDATES         = ["mag_db", "magnitude_db", "mag"]

# Metadados de STFT
COL_STFT_NFFT_CANDIDATES   = ["stft_nfft", "nfft", "NFFT"]
COL_STFT_HOP_CANDIDATES    = ["stft_hop", "hop_length", "hop", "stft_hop_length"]
COL_STFT_WIN_CANDIDATES    = ["stft_window", "window", "win_name", "stft_win"]
COL_SR_CANDIDATES          = ["sr", "sample_rate", "fs"]

# Emoção contínua
COL_VALENCE_CANDIDATES     = ["valence_mean", "valence", "valence_block", "valence_norm"]
COL_AROUSAL_CANDIDATES     = ["arousal_mean", "arousal", "arousal_block", "arousal_norm"]


def _pick_first_existing(columns, candidates):
    """Retorna o primeiro nome de coluna que existir no DataFrame."""
    for c in candidates:
        if c in columns:
            return c
    return None


def plot_fingerprint(
    parquet_path: str,
    song_id=None,
    block_id=None,
    emotion=None,
    max_points: int = 8000,
    save_html: str | None = None
):
    parquet_path = Path(parquet_path)
    print(f"Carregando parquet: {parquet_path}")
    df = pd.read_parquet(parquet_path)

    cols = df.columns.tolist()
    print("Colunas disponíveis:", cols)

    # ---- Descobrir colunas dinamicamente ----
    col_song     = _pick_first_existing(cols, COL_SONG_CANDIDATES)
    col_title    = _pick_first_existing(cols, COL_TITLE_CANDIDATES)
    col_block    = _pick_first_existing(cols, COL_BLOCK_CANDIDATES)
    col_t_anchor = _pick_first_existing(cols, COL_T_ANCHOR_CANDIDATES)
    col_f_anchor = _pick_first_existing(cols, COL_F_ANCHOR_CANDIDATES)
    col_emotion  = _pick_first_existing(cols, COL_EMOTION_CANDIDATES)
    col_mag      = _pick_first_existing(cols, COL_MAG_CANDIDATES)

    col_nfft     = _pick_first_existing(cols, COL_STFT_NFFT_CANDIDATES)
    col_hop      = _pick_first_existing(cols, COL_STFT_HOP_CANDIDATES)
    col_window   = _pick_first_existing(cols, COL_STFT_WIN_CANDIDATES)
    col_sr       = _pick_first_existing(cols, COL_SR_CANDIDATES)

    col_valence  = _pick_first_existing(cols, COL_VALENCE_CANDIDATES)
    col_arousal  = _pick_first_existing(cols, COL_AROUSAL_CANDIDATES)

    if col_t_anchor is None or col_f_anchor is None:
        raise ValueError(
            "Não encontrei colunas de tempo/frequência para os picos.\n"
            f"Tente nomear as colunas como algo em {COL_T_ANCHOR_CANDIDATES} e {COL_F_ANCHOR_CANDIDATES}"
        )

    print(f"Usando coluna de tempo: {col_t_anchor}")
    print(f"Usando coluna de frequência: {col_f_anchor}")

    # ---- Filtros opcionais ----
    if col_song is not None and song_id is not None:
        df = df[df[col_song] == song_id]

    if col_block is not None and block_id is not None:
        df = df[df[col_block] == block_id]

    if col_emotion is not None and emotion is not None:
        df = df[df[col_emotion] == emotion]

    df = df.dropna(subset=[col_t_anchor, col_f_anchor])

    if df.empty:
        raise ValueError("Nenhum dado após os filtros (song/block/emotion).")

    total_points = len(df)
    if total_points > max_points:
        df = df.sample(max_points, random_state=42)
        print(f"Amostrando {len(df)} pontos (de {total_points:,}).")

    # ---- Metadados agregados para o texto ----
    def _safe_scalar(col):
        if col is None or col not in df.columns:
            return None
        values = df[col].dropna()
        if values.empty:
            return None
        return values.iloc[0]

    # ID e título
    song_id_str = str(_safe_scalar(col_song)) if col_song else None
    song_title_str = str(_safe_scalar(col_title)) if col_title else None

    # STFT
    nfft_val   = _safe_scalar(col_nfft)
    hop_val    = _safe_scalar(col_hop)
    win_val    = _safe_scalar(col_window)
    sr_val     = _safe_scalar(col_sr)

    # Valence / arousal
    valence_mean = df[col_valence].mean() if col_valence else None
    arousal_mean = df[col_arousal].mean() if col_arousal else None

    # Quadrante emocional predominante
    quadrant_str = None
    if col_emotion and not df[col_emotion].dropna().empty:
        quadrant_str = df[col_emotion].mode().iloc[0]

    # Bloco (se você filtrou por um bloco específico, use esse; senão, podemos dizer "múltiplos blocos")
    block_info = None
    if block_id is not None:
        block_info = str(block_id)
    elif col_block and not df[col_block].dropna().empty:
        # se houver vários, pega o primeiro só para descritivo
        block_info = str(df[col_block].iloc[0])

    # ---- Monta string principal para dentro da figura ----
    # Título da faixa
    if song_title_str and song_id_str:
        faixa_txt = f"“{song_title_str}” (DEAM ID {song_id_str})"
    elif song_id_str:
        faixa_txt = f"DEAM ID {song_id_str}"
    elif song_title_str:
        faixa_txt = f"“{song_title_str}”"
    else:
        faixa_txt = "Faixa da base DEAM"

    stft_parts = []
    if nfft_val is not None:
        stft_parts.append(f"NFFT = {int(nfft_val)}")
    if hop_val is not None:
        stft_parts.append(f"hop = {int(hop_val)}")
    if win_val is not None:
        stft_parts.append(f"janela {win_val}")
    if sr_val is not None:
        stft_parts.append(f"fs = {int(sr_val)} Hz")

    stft_txt = ", ".join(stft_parts) if stft_parts else "parâmetros de STFT não informados"

    emo_parts = []
    if valence_mean is not None:
        emo_parts.append(f"valence média ≈ {valence_mean:.2f}")
    if arousal_mean is not None:
        emo_parts.append(f"arousal médio ≈ {arousal_mean:.2f}")
    if quadrant_str:
        emo_parts.append(f"quadrante emocional: {quadrant_str}")
    if block_info:
        emo_parts.append(f"bloco: {block_info}")

    emo_txt = "; ".join(emo_parts) if emo_parts else "sem metadados emocionais disponíveis"

    # Texto final para desenhar na figura (multilinha)
    text_inside = (
        f"{faixa_txt}\n"
        f"STFT: {stft_txt}\n"
        f"{emo_txt}"
    )

    print("\n===== TEXTO DINÂMICO PARA A FIGURA =====")
    print(text_inside)
    print("========================================\n")

    # ---- Configura cor ----
    if col_emotion is not None:
        color_arg = col_emotion
    elif col_block is not None:
        color_arg = col_block
    else:
        color_arg = None

    # ---- Configura tamanho ----
    if col_mag is not None:
        mag = df[col_mag].astype(float)
        mag_norm = (mag - mag.min()) / (mag.max() - mag.min() + 1e-9)
        df["_marker_size"] = 4 + 10 * mag_norm
        size_arg = "_marker_size"
    else:
        size_arg = None

    # ---- Cria um ÚNICO scatter ----
    fig = px.scatter(
        df,
        x=col_t_anchor,
        y=col_f_anchor,
        color=color_arg,
        size=size_arg,
        labels={
            col_t_anchor: "Tempo (s)",
            col_f_anchor: "Frequência (Hz)"
        },
        title="Fingerprint espectral",
    )

    fig.update_traces(
        mode="markers",
        marker=dict(
            symbol="circle",
            opacity=0.7,
            line=dict(width=0)
        )
    )

    # Texto dentro da figura (canto inferior esquerdo)
    fig.add_annotation(
        text=text_inside.replace("\n", "<br>"),  # quebra de linha no HTML
        showarrow=False,
        xref="paper",
        yref="paper",
        x=0.01,
        y=-0.20,
        align="left",
        font=dict(size=10),
    )

    fig.update_layout(
        xaxis=dict(title="Tempo (s)", zeroline=False),
        yaxis=dict(title="Frequência (Hz)", zeroline=False),
        template="simple_white",
        legend_title_text="Emoção/Bloco" if color_arg else None,
        height=700,
        margin=dict(b=120)  # espaço extra embaixo para o texto
    )

    if save_html is not None:
        save_html = str(save_html)
        fig.write_html(save_html)
        print(f"Figura salva em: {save_html}")

    fig.show()
    # se não quiser duplicar em alguns ambientes, pode NÃO retornar a figura
    return fig


# ================== EXEMPLO DE USO ==================
plot_fingerprint(PARQUET_PATH, max_points=10000)
# ou, se quiser um bloco específico:
# plot_fingerprint(PARQUET_PATH, block_id=3, max_points=8000)
